#Main Code

In [1]:
!pip install geojson

In [2]:
!pip install scikit-image

In [3]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from numba import jit,cuda
import os
import geojson
import cv2
import skimage
#import torch,torchvision 
#import lmdb

In [4]:
from skimage.draw import polygon,polygon2mask

In [5]:
with open('/content/annot2.geo.json') as f:
  s=f.read()

In [6]:
loader=geojson.loads(s)

In [7]:
loader[0]

{"geometry": {"coordinates": [[[24256.98, 25162.44], [24226.3, 25162.45], [24199.03, 25173.92], [24178.41, 25194.52], [24166.92, 25221.78], [24162.3, 25258.59], [24165.99, 25306.5], [24180.95, 25333], [24203.08, 25354.76], [24245.17, 25373.16], [24272.98, 25373.96], [24291.61, 25369.07], [24295.21, 25366.98], [24307.51, 25361.22], [24318.49, 25353.45], [24318.5, 25353.45], [24326.22, 25345.89], [24332.82, 25337.31], [24338.17, 25327.84], [24342.14, 25317.58], [24344.62, 25306.66], [24345.47, 25295.22], [24344.98, 25286.51], [24343.54, 25278.08], [24341.21, 25270], [24338.03, 25262.31], [24334.07, 25255.07], [24329.37, 25248.33], [24324, 25242.15], [24317.99, 25236.58], [24304.75, 25194.24], [24284.16, 25173.81], [24256.98, 25162.44]]], "type": "Polygon"}, "id": "PathAnnotationObject", "properties": {"classification": {"colorRGB": -14336, "name": "Other"}, "isLocked": false, "measurements": []}, "type": "Feature"}

In [8]:
#len(loader['features'])

In [9]:
#loader['features'][170]

In [10]:
#len(loader['features'][28]['geometry']['coordinates'][1][0])

In [11]:
#len(coordinates)

In [58]:
#np.add(-24150,loader[0]['geometry']['coordinates'][0])
a=loader[0]['geometry']['coordinates'][0]
a

[[24256.98, 25162.44],
 [24284.16, 25173.81],
 [24304.75, 25194.24],
 [24317.99, 25236.58],
 [24324, 25242.15],
 [24329.37, 25248.33],
 [24334.07, 25255.07],
 [24338.03, 25262.31],
 [24341.21, 25270],
 [24343.54, 25278.08],
 [24344.98, 25286.51],
 [24345.47, 25295.22],
 [24344.62, 25306.66],
 [24342.14, 25317.58],
 [24338.17, 25327.84],
 [24332.82, 25337.31],
 [24326.22, 25345.89],
 [24318.5, 25353.45],
 [24318.49, 25353.45],
 [24307.51, 25361.22],
 [24295.21, 25366.98],
 [24291.61, 25369.07],
 [24272.98, 25373.96],
 [24245.17, 25373.16],
 [24203.08, 25354.76],
 [24180.95, 25333],
 [24165.99, 25306.5],
 [24162.3, 25258.59],
 [24166.92, 25221.78],
 [24178.41, 25194.52],
 [24199.03, 25173.92],
 [24226.3, 25162.45],
 [24256.98, 25162.44]]

In [59]:
newa=np.flip(a,axis=1)
newa


array([[25162.44, 24256.98],
       [25173.81, 24284.16],
       [25194.24, 24304.75],
       [25236.58, 24317.99],
       [25242.15, 24324.  ],
       [25248.33, 24329.37],
       [25255.07, 24334.07],
       [25262.31, 24338.03],
       [25270.  , 24341.21],
       [25278.08, 24343.54],
       [25286.51, 24344.98],
       [25295.22, 24345.47],
       [25306.66, 24344.62],
       [25317.58, 24342.14],
       [25327.84, 24338.17],
       [25337.31, 24332.82],
       [25345.89, 24326.22],
       [25353.45, 24318.5 ],
       [25353.45, 24318.49],
       [25361.22, 24307.51],
       [25366.98, 24295.21],
       [25369.07, 24291.61],
       [25373.96, 24272.98],
       [25373.16, 24245.17],
       [25354.76, 24203.08],
       [25333.  , 24180.95],
       [25306.5 , 24165.99],
       [25258.59, 24162.3 ],
       [25221.78, 24166.92],
       [25194.52, 24178.41],
       [25173.92, 24199.03],
       [25162.45, 24226.3 ],
       [25162.44, 24256.98]])

#Important Functions


In [60]:
# reads data
def cords_loader(loader):
  coordinates=[]
  for i in range(len(loader)):
    if loader[i]['geometry']['type']=='Polygon':
      coordinates.append(np.flip(loader[i]['geometry']['coordinates'][0],axis=1))
    #Testing Addition
    elif loader [i]['geometry']['type']=='MultiPolygon':
      for j in range(len(loader[i]['geometry']['coordinates'])):
        coordinates.append(np.flip(loader[i]['geometry']['coordinates'][j][0],axis=1))
  return coordinates

    

In [61]:
#offsets data to avoid negative integers
def cvt_cords2int(coordinates,offset):
  cords_int=[]
  for i in range(len(coordinates)):
    cords_int.append(np.add(offset,coordinates[i]).astype('int32'))
  return cords_int

In [62]:
def int2mask(cords_int,mask_size):
  mask=np.zeros(mask_size,dtype=np.bool)
  for i in range(len(cords_int)):
    mask=np.logical_or(mask,polygon2mask(mask_size,cords_int[i]))
  return mask

In [63]:
def draw_mask(loader,mask_size,offset):
  coordinates=cords_loader(loader)
  cords_int=cvt_cords2int(coordinates,offset)
  return int2mask(cords_int,mask_size)

#Image Display

In [64]:
mask_size=(35999,35844)
offset=0

In [18]:
#ld=loader[:3]
#ld

In [19]:
#maskImg1=np.rot90(draw_mask(ld,(2048,2048),-24150),1)

In [65]:
maskImg=draw_mask(loader,mask_size,0)

In [21]:
#newImg=maskImg.astype(int)

In [66]:
def corr_dim(x,y,tileSize):
  x=x-(x%tileSize)
  y=y-(y%tileSize)
  return (x,y)

In [71]:
def saveFun(i,j,maskImg):
  if not os.path.exists('/content/drive/My Drive/Datasets/WSI/01_tiles_annt'):
    os.mkdir('/content/drive/My Drive/Datasets/WSI/01_tiles_annt')
  cv2.imwrite(f'/content/drive/My Drive/Datasets/WSI/01_tiles_annt/mask{i}_{j}.png', maskImg*255)

In [72]:
@jit
def saveImg(maskImg,tileSize):
  x,y=maskImg.shape
  x,y=corr_dim(x,y,tileSize)
  for i in range(0,x,tileSize):
    for j in range(0,y,tileSize):
      saveFun(i,j,maskImg[i:i+tileSize,j:j+tileSize])

In [69]:
#plt.imshow(maskImg[26880:,26880:],cmap='binary_r')

In [ ]:
saveImg(maskImg,1024)

In [67]:
#plt.imshow(cv2.resize(newImg,(1024,1024)),cmap='binary_r')

In [ ]:
plt.imshow(maskImg1,cmap='binary_r')

In [ ]:
maskImg.shape()

In [85]:
maskImg[1012,106]

False

In [ ]:
cv2.imwrite('/content/mask.png', maskImg * 255)

In [ ]:
#plt.figure(figsize=(10,10))
#plt.imshow(draw_mask(loader,mask_size,offset),cmap='binary_r')
plt.figure(figsize=(10,10))

plt.imshow(np.resize,(1125,1121)),cmap='binary_r')

In [74]:
print(len(os.listdir('/content/drive/My Drive/Datasets/WSI/01_tiles_annt')))
len(os.listdir('/content/drive/My Drive/Datasets/WSI/01_tiles'))

1225


1225

#Commented

In [ ]:
!pip install --upgrade setuptools pip
!pip install openslide
!apt update && apt install -y openslide-tools
!pip install openslide-python
!pip install py_wsi

In [ ]:
!pip install numba

In [ ]:



!git clone https://github.com/cosmiq/solaris.git
!cd /content/solaris

In [ ]:
#!pip install -r /content/solaris/requirements.txt

In [ ]:
!apt install gdal-bin python-gdal python3-gdal --quiet
!apt install python3-rtree --quiet
!apt install libgdal-dev libproj-dev --quiet
!apt install libgeos-dev
!pip install git+git://github.com/geopandas/geopandas.git --quiet
!pip install descartes --quiet
!pip install splot hvplot geoviews --quiet

In [ ]:
!apt install python-cartopy python3-cartopy --quiet
!apt install gdal-bin python-gdal python3-gdal --quiet
!apt install python3-rtree --quiet
!apt install libgdal-dev libproj-dev --quiet
!apt install libgeos-dev
!pip install git+git://github.com/geopandas/geopandas.git --quiet
!pip install descartes --quiet
!pip install urbanaccess pandana pysal --quiet
!pip install splot hvplot geoviews --quiet
!pip install --no-binary shapely shapely --upgrade --force-reinstall --quiet

#Commented

In [ ]:
#generates an empty image

In [ ]:
#pg=polygon(cords_int[0][:,0],cords_int[0][:,1])
#pg[1]


In [ ]:
#cords_int[0][:,0]

In [ ]:
# combines all masks
#for i in range(len(cords_int)):
  #pg=polygon(cords_int[i][:,0],cords_int[i][:,1])
 # mask=np.logical_or(mask,polygon2mask((1024,1024),cords_int[i]))

In [ ]:
#polygon2mask((512,512),cords_int[0])

In [ ]:
#mask=draw_mask(loader)

In [ ]:
#plt.imshow(mask,cmap='binary_r')

In [ ]:
'''def draw_mask(loader):
  coordinates=[]
  # reads data
  for i in range(len(loader['features'])):
    if loader['features'][i]['geometry']['type']=='Polygon':
      coordinates.append(loader['features'][i]['geometry']['coordinates'][0])
    #Testing Addition
    elif loader['features'][i]['geometry']['type']=='MultiPolygon':
      for j in range(len(loader['features'][i]['geometry']['coordinates'])):
        coordinates.append(loader['features'][i]['geometry']['coordinates'][j][0])
  #offsets data to avoid negative integers
  cords_int=[]
  for i in range(len(coordinates)):
    cords_int.append(np.add(200,coordinates[i]).astype('int32'))
  #generates black image
  mask=np.zeros((512,512),dtype=np.bool)
  # combines all masks
  for i in range(len(cords_int)):
    mask=np.logical_or(mask,polygon2mask((512,512),cords_int[i]))
  return mask'''

"def draw_mask(loader):\n  coordinates=[]\n  # reads data\n  for i in range(len(loader['features'])):\n    if loader['features'][i]['geometry']['type']=='Polygon':\n      coordinates.append(loader['features'][i]['geometry']['coordinates'][0])\n    #Testing Addition\n    elif loader['features'][i]['geometry']['type']=='MultiPolygon':\n      for j in range(len(loader['features'][i]['geometry']['coordinates'])):\n        coordinates.append(loader['features'][i]['geometry']['coordinates'][j][0])\n  #offsets data to avoid negative integers\n  cords_int=[]\n  for i in range(len(coordinates)):\n    cords_int.append(np.add(200,coordinates[i]).astype('int32'))\n  #generates black image\n  mask=np.zeros((512,512),dtype=np.bool)\n  # combines all masks\n  for i in range(len(cords_int)):\n    mask=np.logical_or(mask,polygon2mask((512,512),cords_int[i]))\n  return mask"

#Commented

In [ ]:

coordinates=loader['features'][0]['geometry']['coordinates'][0]
np_cords=np.array(coordinates)

In [ ]:
cords_int=np_cords.astype('int32')

In [ ]:
cords_int[:5]

array([[61, 35],
       [62, 35],
       [62, 35],
       [63, 35],
       [63, 36]], dtype=int32)

In [ ]:
pg=polygon(cords_int[:,1],cords_int[:,0])

In [ ]:
pg

(array([29, 29, 29, 29, 29, 29, 30, 30, 30, 30, 30, 31, 31, 31, 31, 31, 31,
        31, 31, 31, 32, 32, 32, 32, 32, 32, 32, 32, 32, 33, 33, 33, 33, 33,
        33, 33, 33, 33, 33, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 35,
        35, 35, 35, 35, 35, 35, 35, 36, 36, 36, 36, 36, 36, 36, 36, 36, 37]),
 array([60, 61, 62, 63, 64, 65, 61, 62, 63, 64, 65, 60, 61, 62, 63, 64, 65,
        66, 67, 68, 60, 61, 62, 63, 64, 65, 66, 67, 68, 60, 61, 62, 63, 64,
        65, 66, 67, 68, 69, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 63,
        64, 65, 66, 67, 68, 69, 70, 64, 65, 66, 67, 68, 69, 70, 72, 73, 70]))

In [ ]:
#pol=np.array([[2,3],[3,1],[4,0]])
mask=polygon2mask((256,256),cords_int[0])

In [ ]:
mask

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])